<a href="https://colab.research.google.com/github/AnkitaSK/NamasteBot/blob/main/NamasteBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unstructured[pdf] PyPDF2 camelot Pillow sentence-transformers chromadb google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 70.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 100.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of sqlalc

In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 104.8 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 0.7.10
    Uninstalling SQLAlchemy-0.7.10:
      Successfully uninstalled SQLAlchemy-0.7.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
camelot 12.6.29 requires SQLAlchemy<0.8.0,>=0.7.7, but you have sqlalchemy 2.0.37 which is incompatible.


In [3]:
from PyPDF2 import PdfReader
import camelot
from PIL import Image
from sentence_transformers import SentenceTransformer
from chromadb import Client
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import google.generativeai as genai
import io
import zlib
import imageio.v3 as iio

In [4]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (190 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
!apt-get update
!apt-get install -y tesseract-ocr

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,654 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,606 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,651

In [6]:
#!tesseract --version

In [7]:
!pip install pytesseract

In [8]:
import pytesseract

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

## Extract data

In [10]:
from unstructured.partition.pdf import partition_pdf


pdf_file_name = "Haridwar-Travel-Guide-by-ixigo.com.pdf"
pdf_folder_path = "/content/data/images/"

raw_pdf_elements = partition_pdf(
    filename=pdf_file_name,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=pdf_folder_path,
)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
set([str(type(el)) for el in raw_pdf_elements])

In [11]:
elements = raw_pdf_elements[3].metadata.orig_elements
chunk_images = [el for el in elements if 'Image' in str(type(el))]
chunk_images[0].to_dict()

{'type': 'Image',
 'element_id': '608d96a4-b5e1-427f-8374-958e70f54fbf',
 'text': '',
 'metadata': {'coordinates': {'points': ((72.22222222222221,
     878.2222222222221),
    (72.22222222222221, 1267.8055555555554),
    (591.6666666666666, 1267.8055555555554),
    (591.6666666666666, 878.2222222222221)),
   'system': 'PixelSpace',
   'layout_width': 2339,
   'layout_height': 1653},
  'last_modified': '2025-02-04T20:55:42',
  'filetype': 'PPM',
  'languages': ['eng'],
  'page_number': 3,
  'image_path': '/content/figures/figure-3-19.jpg'}}

In [12]:
# Categorize extracted elements from a PDF into tables and texts.
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))


In [13]:
print(texts)

["ixigo\n\nHaridwar Travel Guide - http://www.ixigo.com/travel-guide/haridwar\n\nHaridwar\n\nRe\n\nOne of the seven holiest places for Hindus, Haridwar literally means a 'gateway to God', succinctly capturing its pious and spiritual character. Come here for an epiphanic union with the almighty, or just sit by the scintillating river Ganga and watch your stress float away with the swishing water and a cool, balming breeze.\n\n”\n\né&\n\né&\n\nThis Pdf and its contents are copyright © 2013,ixigo.com, all rights reserved. ixigo logo, the character and all other marks displayed herein are registered and/or common law trademarks of ixigo.com and/or third parties.\n\nFam ous For\n\nPlces To VsitRelyDusCity\n\nThe holy town of Haridwar marks the arrival of the River Ganga into the Northern plains, the first time since its origin at Gaumukh, Gangotri. Visitors hail from all across the country, and globe, for a dip in the holy waters of the river, which is believed to wash away one's sins and h

In [ ]:
print(tables)

In [14]:
# Get the images from the CompositeElement objects
def get_images(chunks):
    images = []
    for chunk in chunks:
        if "CompositeElement" in str(type(chunk)):
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                # print(dir(el.metadata))
                if "Image" in str(type(el)):
                    images.append(el.metadata.image_path)
    return images

images = get_images(raw_pdf_elements)

In [15]:
images[0]

'/content/figures/figure-1-1.jpg'

In [16]:
import base64
from IPython.display import Image, display

def display_base64_image(base64_code):
    # Decode the base64 string to binary
    image_data = base64.b64decode(base64_code)
    # Display the image
    display(Image(data=image_data))

# display_base64_image(images_base64[25])

In [17]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00


In [18]:
from langchain.text_splitter import CharacterTextSplitter
# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=10000, chunk_overlap=0
)
joined_texts = " ".join(texts)
texts_4k_token = text_splitter.split_text(joined_texts)

In [ ]:
# def encode_image(image_path: str) -> str:
#     """Getting the base64 string"""
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode("utf-8")


## Use Gemini API

In [19]:
import google.generativeai as genai
from google.colab import userdata
import os

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## Summarize the data

# Text summary

In [20]:
MODEL_NAME = "models/gemini-1.5-pro-latest"
model = genai.GenerativeModel(model_name=MODEL_NAME)

def make_prompt(element):
  return f"""You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """

def generate_text_summaries(texts, tables, summarize_texts=False):
  """
    Summarize text elements

    Args:
      texts: List of str
      tables: List of str
      summarize_texts: Bool to summarize texts
  """

  text_summaries, table_summaries = [], []
  # print(dir(model))

  if texts and summarize_texts:
    text_summaries = [
            model.generate_content(make_prompt(text)).candidates[0].content.parts[0].text
            for text in texts
        ]

  if tables:
    table_summaries = [
            model.generate_content(make_prompt(table)).candidates[0].content.parts[0].text
            for table in tables
        ]

  return text_summaries, table_summaries


In [21]:
text_summaries, table_summaries = generate_text_summaries(texts, tables, summarize_texts=True)

In [22]:
text_summaries

['Haridwar, a Hindu holy city, is situated where the Ganges River enters the Northern plains.  Known for Har ki Pauri, a sacred bathing ghat, and the evening aarti ceremony, it attracts pilgrims seeking spiritual cleansing.  Key sites include Chandi Devi Temple, Maya Devi Temple, and Mansa Devi Temple.  Haridwar also hosts the Kumbh Mela.  Rajaji National Park offers wildlife viewing.  Bara Bazaar is a popular market.  The city maintains a vegetarian and alcohol-free environment.\n',
 'Haridwar monthly weather and Har Ki Pauri highlight.  Jan-May and Nov-Dec: Cold, heavy woolens. June-Oct: Pleasant, light woolens, umbrella (except Oct).  Har Ki Pauri: Holy Ghat, Ganga touches plains, Vishnu footprint, evening Aarti with floating lamps.\n',
 'Haridwar temples and ghats: Mansa Devi Temple, accessed by ropeway or trek, fulfills wishes. Ganesh Ghat, near Hanuman Temple, offers Ganga views, Dam Kothi, photography spots. Mercury Temple (Paradeshwar Mahadev) features a mercury shivalinga.\n',

In [ ]:
table_summaries

# Image summary

In [23]:
# also called encoding of image
def image_to_base64(image_path):
    # Open the image file in binary mode
    with open(image_path, "rb") as image_file:
        # Read the image content and encode it in base64
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_image



In [ ]:
image_path = "/content/figures/figure-1-1.jpg"
image_base64 = image_to_base64(image_path)
image_base64

In [24]:
def summarize_image(base64_image):
    """Generate a summary from an image using Gemini API."""
    response = model.generate_content(
        contents=[
            {
                "role": "user",
                "parts": [
                    {"text": "Summarize the content of this image in a few sentences."},
                    {
                        "inline_data": {
                            "mime_type": "image/jpeg",  # Change based on image type
                            "data": base64_image
                        }
                    }
                ]
            }
        ]
    )

    return response.candidates[0].content.parts[0].text


def generate_image_summaries(images_base64):
    """Generate summaries for a list of images."""
    image_summaries = []
    for image_base64 in images_base64:
        summary = summarize_image(image_base64)
        image_summaries.append(summary)

    return image_summaries


In [25]:
# image_summary = summarize_image(image_base64)
images_base64 = [image_to_base64(image) for image in images]
print(images_base64[3])

/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAEIAQgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3+iiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiue8R679hje1gJ84rl3HO0HsPeplJRV2NK7sbi3MDuyLPGzqcFQ4JF

In [26]:
image_summaries = generate_image_summaries(images_base64)

In [27]:
image_summaries

['The image shows the logo for ixigo.com. The logo consists of the word "ixigo" in a stylized font, with the "x" in orange and the rest of the letters in purple.  ".com" is included in smaller purple letters below.\n',
 'The image shows a pair of grey quotation marks.  They are slightly angled and appear to be part of a larger text element, likely indicating a quote or dialogue.\n',
 'The image depicts a simple, curved line. It starts thin at the top left, thickens slightly in the middle, and tapers off again at the bottom right. The curve is smooth and consistent, resembling a segment of a circle or ellipse.',
 "A cartoon male character, dressed as a tour guide or explorer, is depicted. He wears a khaki outfit, including a hat, vest, and shorts, along with hiking boots and glasses.  He's smiling and pointing with his right hand, possibly directing attention to something unseen. He also appears to be holding a small orange device, perhaps a camera or a whistle, in his left hand.\n",
 "

## Store in vector db - Chroma

In [28]:
import chromadb
from sentence_transformers import SentenceTransformer

In [30]:
# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# # Create a collection for storing summaries
# collection = chroma_client.get_or_create_collection(name="summaries")
# Create a collection for storing summaries with embeddings
collection = chroma_client.get_or_create_collection(name="summaries_with_embeddings")

# Load Sentence-Transformers model
embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
def embedding_common(summary):
 return embedding_model.encode(summary).tolist()

In [32]:
def store_summary_in_chroma(id, summary, metadata):
    """Store summary in ChromaDB."""
    collection.add(
        ids=[id],
        documents=[summary],
        metadatas=[metadata],
        embeddings=[embedding_common(summary)]
    )

In [33]:
# Store Summaries in ChromaDB
for i, summary in enumerate(text_summaries):
    store_summary_in_chroma(f"text_{i}", summary, {"type": "text"})

if table_summaries:
  for i, summary in enumerate(table_summaries):
      store_summary_in_chroma(f"table_{i}", summary, {"type": "table"})

for i, summary in enumerate(image_summaries):
    store_summary_in_chroma(f"image_{i}", summary, {"type": "image"})

In [34]:
def retrieve_similar_summaries(query):
    """Retrieve the most relevant stored summaries."""
    results = collection.query(query_texts=[query], n_results=3)
    return results["documents"]


In [35]:
# Example Retrieval Query
query = "Tell me about a holy city"
retrieved_summaries = retrieve_similar_summaries(query)
print("🔍 Retrieved Summaries:", retrieved_summaries)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 34.7MiB/s]


🔍 Retrieved Summaries: [['Haridwar, a Hindu holy city, is situated where the Ganges River enters the Northern plains.  Known for Har ki Pauri, a sacred bathing ghat, and the evening aarti ceremony, it attracts pilgrims seeking spiritual cleansing.  Key sites include Chandi Devi Temple, Maya Devi Temple, and Mansa Devi Temple.  Haridwar also hosts the Kumbh Mela.  Rajaji National Park offers wildlife viewing.  Bara Bazaar is a popular market.  The city maintains a vegetarian and alcohol-free environment.\n', 'Haridwar travel guide featuring Mercury Temple (selling rudraksha beads), Rajaji National Park (diverse flora/fauna, Ganges river flows through it, home to elephants, tigers, leopards), and 5 restaurants:  Dosa Plaza (South Indian food near railway station), Big Ben Restaurant (veg thalis, near railway/bus station), Invitation (multi-cuisine in modern Haridwar), Fun n Food (snacks, popular hangout), and Hoshiyar Puri.\n', 'Haridwar temples and ghats: Mansa Devi Temple, accessed by 

## TODO setup multimodel embedding models
#### Need an image embedding model (like CLIP) to generate embeddings for images and a text embedding model (like sentence-transformers) for text.



# RAG Pipeline

In [36]:
def retrieve_document(query):
    """Retrieve the most relevant document based on the query."""
    # Generate embedding for the query (text or image)
    query_embedding = embedding_common(query)

    # Query ChromaDB for the most similar document based on the query embedding
    results = collection.query(query_embeddings=[query_embedding], n_results=3)

    return results['documents'], results['metadatas']

In [37]:
def generate_response(retrieved_documents):
    """Generate a response based on the retrieved documents."""
    prompt = f"""
    You are an assistant tasked with summarizing information. Here are the retrieved documents:

    {retrieved_documents}

    Provide a concise and informative summary based on the information above. No extra information should be provided.
    """

    # Generate the content using Gemini 1.5 Pro
    response = model.generate_content(prompt)

    # Accessing the content from the response object
    if hasattr(response, 'candidates') and response.candidates:
        candidates = response.candidates
        if len(candidates) > 0:
            generated_response = candidates[0].content.parts[0].text
            return generated_response
    return "No content generated. Please check the input."

In [38]:
def rag_pipeline(query, is_text_query=True):
    """Complete RAG pipeline: retrieve and generate response."""

    # Step 1: Retrieve the most relevant documents (text or image)
    retrieved_documents, _ = retrieve_document(query)

    # Step 2: Generate the response based on the retrieved documents
    generated_response = generate_response(retrieved_documents)

    return generated_response

In [39]:
# Example Text Query
query = "Tell me about a holy city near the Ganges River."
result = rag_pipeline(query)
print(result)


Haridwar, a Hindu holy city where the Ganges enters the Northern plains, is known for Har ki Pauri ghat, evening aarti ceremonies, and temples like Chandi Devi, Maya Devi, and Mansa Devi.  It hosts the Kumbh Mela and offers Rajaji National Park for wildlife viewing, along with Bara Bazaar for shopping. The city maintains a vegetarian and alcohol-free environment.  Crowds gather along the riverbanks for rituals and bathing, especially during dawn or dusk.



In [ ]:
query = "List restaurants"
result = rag_pipeline(query)
print(result)

# With translation

In [40]:
pip install langdetect

In [41]:
pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00


In [42]:
from langdetect import detect
from deep_translator import GoogleTranslator
import google.generativeai as genai

In [43]:
# Function to detect language
def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except Exception as e:
        print("Language detection error:", e)
        return "en"  # Default to English if detection fails

In [44]:
# Function to translate text to English
def translate_to_english(text, src_lang):
    if src_lang != 'en':  # Only translate if the source language is not English
        return GoogleTranslator(source=src_lang, target='en').translate(text)
    return text

In [45]:
# Function to translate text back to the original language
def translate_back(text, target_lang):
    if target_lang != 'en':  # Only translate back if the target language is not English
        return GoogleTranslator(source='en', target=target_lang).translate(text)
    return text

In [48]:
# Updated RAG pipeline to support language translation
async def rag_pipeline_with_language_support(query):
    # Step 1: Detect language of the query
    detected_lang = detect_language(query)

    # Step 2: Translate query to English if needed
    translated_query = translate_to_english(query, detected_lang)

    # Step 3: Retrieve relevant documents for the translated query
    retrieved_summaries = retrieve_similar_summaries(translated_query)

    # Step 4: Generate a response from the retrieved summaries (in English)
    response = generate_response(retrieved_summaries)

    # Step 5: Translate response back to the original language
    final_response = translate_back(response, detected_lang)

    return final_response

In [49]:
# Example usage1 Hindi
query = "गंगा नदी के पास एक पवित्र शहर के बारे में बताएं"
response = await rag_pipeline_with_language_support(query)
print(response)

गंगा नदी पर बसा पवित्र शहर हरिद्वार हर की पौड़ी घाट और शाम की आरती के लिए जाना जाता है। तीर्थयात्री आध्यात्मिक शुद्धि और कुंभ मेले के लिए आते हैं। प्रमुख मंदिरों में चंडी देवी, माया देवी और मनसा देवी (रोपवे/ट्रेक द्वारा पहुँचा जा सकता है) शामिल हैं। राजाजी राष्ट्रीय उद्यान वन्यजीवों को देखने का अवसर प्रदान करता है। बड़ा बाज़ार एक लोकप्रिय बाज़ार है। शहर शाकाहारी और शराब-मुक्त है। अन्य उल्लेखनीय स्थलों में बुध मंदिर (पारदेश्वर महादेव) और गणेश घाट शामिल हैं।


In [50]:
# Example usage1 Spanish
query = "Habla sobre una ciudad santa cerca del río Ganges."
response = await rag_pipeline_with_language_support(query)
print(response)

Haridwar, una ciudad sagrada donde el Ganges ingresa a las llanuras del norte, es conocida por el ghat Har ki Pauri y su ceremonia aarti vespertina. Los peregrinos la visitan para purificarse espiritualmente y ver lugares como los templos de Chandi Devi, Maya Devi y Mansa Devi. Haridwar es sede del Kumbh Mela. Las atracciones cercanas incluyen el Parque Nacional Rajaji (vida silvestre, río Ganges) y el Bazar Bara. La ciudad mantiene un ambiente vegetariano y sin alcohol. Los templos clave incluyen Mansa Devi (accesible por teleférico) y el Templo de Mercurio (shivalinga de mercurio). Los restaurantes populares incluyen Dosa Plaza, Big Ben, Invitation, Fun n Food y Hoshiyar Puri.


In [51]:
# Example usage1 English
query = "Tell about a holy city near river Ganga."
response = await rag_pipeline_with_language_support(query)
print(response)

Haridwar, a holy city where the Ganges enters the Northern plains, is known for Har ki Pauri ghat and its evening aarti.  Pilgrims visit for spiritual cleansing at sites like Chandi Devi, Maya Devi, and Mansa Devi Temples.  Haridwar hosts the Kumbh Mela.  Rajaji National Park offers wildlife viewing.  Bara Bazaar is a popular market. The city maintains a vegetarian and alcohol-free environment.  Notable sites include the Mercury Temple and various ghats.  Food options range from South Indian at Dosa Plaza to multi-cuisine at Invitation and vegetarian thalis at Big Ben, near the railway/bus station.

